#### 先说说 HashMap 原理吧？
HashMap主要由数组和链表组成，他不是线程安全的。核心的点就是put插入数据的过程，get查询数据以及扩容的方式。JDK1.7和1.8的主要区别在于头插和尾插方式的修改，头插容易导致HashMap链表死循环，并且1.8之后加入红黑树对性能有提升。

**1. put 插入数据流程**     
往map插入元素的时候首先通过对key hash然后与数组长度-1进行与运算((n-1)&hash)，都是2的次幂所以等同于取模，但是位运算的效率更高。找到数组中的位置之后，  
(1) 如果数组中没有元素直接存入，   
(2) 反之则判断key是否相同，key相同就覆盖，否则就会插入到链表的尾部，如果链表的长度超过8，则会转换成红黑树，最后判断数组长度是否超过默认的长度*负载因子也就是 12，超过则进行扩容。  

**2. get 查询数据**
查询数据相对来说就比较简单了，首先计算出hash值，然后去数组查询，是红黑树就去红黑树查，链表就遍历链表查询就可以了。

**3. resize 扩容过程**
扩容的过程就是对key重新计算hash，然后把数据拷贝到新的数组。**


#### 那多线程环境怎么使用 Map 呢？ConcurrentHashmap 了解过吗？

#### 1.7分段锁
从结构上说，1.7版本的ConcurrentHashMap采用分段锁机制，里面包含一个Segment数组，Segment继承于ReentrantLock，Segment则包含HashEntry的数组，HashEntry本身就是一个链表的结构，具有保存key、value的能力能指向下一个节点的指针。实际上就是相当于每个Segment都是一个HashMap，默认的Segment长度是16，也就是支持16个线程的并发写，Segment之间相互不会受到影响。


**1. put 流程**

其实发现整个流程和HashMap非常类似，只不过是先定位到具体的Segment，然后通过ReentrantLock去操作而已，后面的流程我就简化了，因为和HashMap基本上是一样的。
(1) 计算hash，定位到segment，segment如果是空就先初始化    
(2) 使用ReentrantLock加锁，如果获取锁失败则尝试自旋，自旋超过次数就阻塞获取，保证一定获取锁成功     
(3) 遍历HashEntry，就是和HashMap一样，数组中key和hash一样就直接替换，不存在就再插入链表，链表同样      


**2. get 流程**
get也很简单，key通过hash定位到segment，再遍历链表定位到具体的元素上，需要注意的是value是volatile的，所以get是不需要加锁的。      


#### 1.8CAS+synchronized
1.8抛弃分段锁，转为用CAS+synchronized来实现，同样HashEntry改为Node，也加入了红黑树的实现。主要还是看put的流程。

**1. put 流程**

(1) 首先计算hash，遍历node数组，如果node是空的话，就通过CAS+自旋的方式初始化      
(2) 如果当前数组位置是空则直接通过CAS自旋写入数据
(3) 如果hash==MOVED，说明需要扩容，执行扩容
(4) 如果都不满足，就使用synchronized写入数据，写入数据同样判断链表、红黑树，链表写入和HashMap的方式一样，key hash一样就覆盖，反之就尾插法，链表长度超过8就转换成红黑树      

**2. get 查询**
get很简单，通过key计算hash，如果key hash相同就返回，如果是红黑树按照红黑树获取，都不是就遍历链表获取。    
get 操作是全程不加锁的, 因为 get 主要是计算 hash 找到对应的 node 数组中的位置后, 进行树/链表的遍历, 树/链表节点表现为 Node 类, 该类的 next 指针和 val 元素都是 volatile 的, 所以不用加锁.

 